1. Imoprt users list.  Broke up the ~125,000 valid users from the previous Jupyter notebook into 5 json files to help create more manageable datasets and to increase productivity with multiple people running the code without risking overlapping users.

In [2]:
import os
import json

folder = os.path.join('C:/', 'users', 'charr', 'documents')
with open(os.path.join(folder, 'coachella_users_5.json'), mode = 'r', encoding = 'utf-8') as reader:
    all_user_list = list(json.load(reader))

In [5]:
len(all_user_list)

13077

In [6]:
from twython import Twython
from twython import TwythonError
from twython import TwythonRateLimitError
import dateutil.parser as dateparser
from time import sleep
from pprint import pprint

consumer_key = 'Xs4Xzym3HuF5izdfCh6fbbbh6'
consumer_secret = 'dyVs6OtGCqlSaBUAl4J1g0c9u29K5Xm3kKLee8O2Vvf2mo1T7P'
access_token = '970138235514114048-jRtzTNNUGdGW7K1Uqvc48bok2zqpROa'
access_token_secret = 'tMWPX4Uq1sQJNClWyLePprvllrY43ftNUV9o6AvrZt9Wb'


twitter = Twython(consumer_key, consumer_secret, access_token, access_token_secret)

2. using the timestamped snowflaked tweet ID as the max_id for all users you can start at the point in the user's tweet history when coachella ended to speed up the process (so rather than go back 200 tweets each request until we either did or did not reach the coachella time frame we cut that process out) and then can collect only the tweets that were in the created_at date range (so you don't go outside the coachella date range)

In [7]:
coachella_end_tweet_id = 1120182057383022594
coachella_start_tweet_id = 1116479450026106880

In [8]:
import datetime

def get_tweet_timestamp(tid):
    offset = 1288834974657
    tstamp = (tid >> 22) + offset
    utcdttime = datetime.datetime.fromtimestamp(tstamp/1000, tz=datetime.timezone.utc)
    return utcdttime
    

In [9]:
coachella_start_date = get_tweet_timestamp(1116479450026106880)
coachella_loose_end_date = get_tweet_timestamp(1120182057383022594)

3. functions work to get tweet history from each user.  We want to make sure that the tweet mode is 'extended' or else if the tweet is longer than 140 characters we won't get the whole tweet, and we can call a max of 200 tweets per request (1 request per second).  in the get_first_tweets_from_user() function, the max_id is set to the coachella end date tweet ID so we don't have to pull more tweets than necessary, and if we need to go back past 200 tweets for a single user the get_max_id() function feeds into the get_tweets_from_user() function. Since twitter requests are limited to 1 per second we also put a sleep timer after every call to guarantee that we won't exceed the request limit.  in the get_tweets() function we also added progress reports to check our progress

In [10]:
def get_first_tweets_from_user(username):
    initial_tweet_list =  twitter.get_user_timeline(screen_name = username, count = 200, tweet_mode='extended', max_id = coachella_end_tweet_id)
    return initial_tweet_list

def get_max_id(tweet_list):
    maxid = tweet_list[-1]['id']
    return maxid

def get_tweets_from_user(username, maxid):
    lst =  twitter.get_user_timeline(screen_name = username, count = 200, max_id = maxid, tweet_mode='extended')
    return lst

def get_tweet_time(tweet_dict):
    return dateparser.parse(tweet_dict['created_at'])

def get_tweet_id_time(tweet_dict):
    return tweet_dict['id'], get_tweet_time(tweet_dict)

def get_tweets(user, earliest_time, latest_time):
    user_tweets = dict()
    user_tweet_group = get_first_tweets_from_user(user)
    
    while True:
        if not user_tweet_group:
            print('3,200 tweet limit reached before', latest_time.strftime('%m/%d/%y'))
            break
        
        min_id, min_datetime = get_tweet_id_time(user_tweet_group[-1])
        if min_datetime > latest_time:
            #print('Searching for tweet group,', min_datetime.strftime('%m/%d/%y'),'..')
            print('.', sep = '', end = '')
            
        else:
            max_id, max_datetime = get_tweet_id_time(user_tweet_group[0])
            if max_datetime < earliest_time:
                print('Found tweet before range', max_datetime.strftime('%m/%d/%y'))
                break
                
            for tweet in user_tweet_group:
                time = get_tweet_time(tweet)
                if earliest_time <= time <= latest_time:
                    user_tweets[tweet['id']] = tweet
                    
            if max_id == min_id:
                break
                
        user_tweet_group = get_tweets_from_user(user, min_id)[1:]
        sleep(1)

    else:
        print('3,200 tweet limit reached before', latest_time.strftime('%m/%d/%y'))
        
    sleep(1)
    
    return list(user_tweets.values())

            

4.  ran the get_tweets() function on all users in teh all_user_list group.  once the length of the user_tweet_dct reaches 1000 the dict is exported to a json file, the old dict is cleared and the process is continued.  if we need to pause the query in the middle we also added a separate dictionary of all users who had been searched for previously  so that there is no risk of overlap.  We also kept the try/except catch in case any of the users had updated their status to private or been banned or anything else between the time we ran the original get_valid_users() function and now. 

In [12]:
#######user_tweet_dct = {}
#####finished_names = {}

In [13]:
def save_tweet_dict(dct):
    file_suffix = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
    filename = 'coachella_tweets' + file_suffix + '.json'
    filepath = os.path.join(folder, filename)
    with open(filepath, mode='w', encoding = 'utf-8') as f:
        json.dump(dct, f)

In [2]:
unknown_users = [z for z in all_user_list if (z not in user_tweet_dct) and (z not in finished_names)]

for user in unknown_users:
    if len(user_tweet_dct) > 1000:
        save_tweet_dict(user_tweet_dct)
        user_tweet_dct = {}
        
    try:
        user_tweet_dct[user] = get_tweets(user, coachella_start_date, coachella_loose_end_date)
        finished_names[user] = True
    except TwythonError:
        user_tweet_dct[user] = []
        finished_names[user] = True
        continue
        

In [ ]:
#final save_tweet_dict for last group of <1000 users
save_tweet_dict(user_tweet_dct)